In [1]:
import torch
import pandas as pd
import re
import os
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='russian')
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv(r"C:\Users\coolz\assignments\data.csv", names = ['Review', 'Rating'], sep=',', dtype = {"Review": str, 'Rating': 'int32' } )
#df = df.drop_duplicates
df = df.dropna()
np.random.seed(37)
df.drop_duplicates(subset=['Review', 'Rating'], keep=False)


,Review,Rating
14,"Мне показалось, что телефон очень прочный, рон...",5
26,Пользовался около месяца. С точки зрения эргон...,3
36,"Сразу скажу,что это не отзыв а совет людям,кот...",5
38,Возвращаюсь к Sony после нескольких эксперимен...,5
40,"В целом, за 300 с небольшим баксов вполне вмен...",4
...,...,...
458426,"удобный, всё работает отлично, звонит, играет,...",5
458427,"прошло больше года, притензий нет, при моей на...",5
458428,мой первый аппарат на андроиде. На данный моме...,5
458429,Разбил iphone и не было желания покупать новый...,5


In [3]:
df1 = df.sample(frac = 0.3)

In [4]:
df1 = df1.applymap(lambda s:s.lower() if type(s) == str else s)

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [6]:
df1["no_punkt"] = df1['Review'].str.replace('[^\w\s]','')

C:\Users\coolz\AppData\Local\Temp/ipykernel_18020/3057402982.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1["no_punkt"] = df1['Review'].str.replace('[^\w\s]','')


In [7]:
print (df.columns.tolist())

['Review', 'Rating']


In [8]:
stop_words = stopwords.words('russian')
df1['clean'] = df1['no_punkt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [9]:
#df1['tokenized_reviews'] = df1.apply(lambda row: nltk.word_tokenize(row['clean']), axis=1)

In [10]:
#df1['stemmed'] = df1.apply(lambda row: [stemmer.stem(item) for item in row.clean], axis=1)

In [11]:
X = df1['clean']
y = df1['Rating']

In [12]:
#df1

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                        test_size = 0.3, random_state=24)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
#Vectorizing the text data
ctmTr = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)
from sklearn.linear_model import LogisticRegression
#Training the model
lr = LogisticRegression(solver='lbfgs', max_iter=500)
lr.fit(ctmTr, y_train)
#Accuracy score
lr_score = lr.score(X_test_dtm, y_test)
print("Accuracy")
print(lr_score)
#Predicting the labels for test data
y_pred_lr = lr.predict(X_test_dtm)
from sklearn.metrics import confusion_matrix
#Confusion matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_lr).ravel()
print(tn, fp, fn, tp)
#True positive and true negative rates
tpr_lr = round(tp/(tp + fn), 4)
tnr_lr = round(tn/(tn+fp), 4)
print(tpr_lr, tnr_lr)

Accuracy
0.629811192544775


ValueError: too many values to unpack (expected 4)